# Different Structure Hybrid Recommender

In [1]:
def init_kaggle(run_on_kaggle, secret_name):
    if run_on_kaggle:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        personal_token = user_secrets.get_secret(secret_name)
        
        import subprocess
        subprocess.run(["git", "clone", "https://" + personal_token + "@github.com/alecontuIT/rec_sys_challenge2022.git"])
        
        import os
        os.chdir("./rec_sys_challenge2022")
        
        subprocess.run(["pip", "install", "-r", "requirements.txt"])
        return True
    
    else:
        return False

In [2]:
dataset_version = "interactions-all-ones"
fast = True
kaggle = init_kaggle(False, "recsys_git_token")

In [3]:
import utils
from recmodels import *
from Recommenders.Hybrids.FastLoadWrapperRecommender import FastLoadWrapperRecommender
from Recommenders.Hybrids.DiffStructHybridRecommender import DiffStructHybridRecommender
from HyperparameterTuning.IterativeHybridOptimizer import DiffStructHybridOptimizer
from scipy.stats import loguniform
from Evaluation.Evaluator import EvaluatorHoldout
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

Tensorflow is not available
Tensorflow is not available


/Users/michael/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [4]:
recommender_class = DiffStructHybridRecommender

In [5]:
URM_all, URM_train, URM_val, ICM = utils.get_data_global_sample(dataset_version=dataset_version, 
                                                                train_percentage=0.7,
                                                                setSeed=True)

## Optimizer

In [9]:
optimizer = DiffStructHybridOptimizer(URM_train, 
                                      URM_val, 
                                      [#TopPopRec, 
                                       #ItemKNNCFRec, 
                                       #UserKNNCFRec, 
                                       #IALSRec,
                                       #SLIM_BPRRec, 
                                       P3AlphaRec, 
                                       RP3BetaRec, 
                                       EASE_R_Rec, 
                                       #MatrixFactorizationBPRRec, 
                                       #FunkSVDRec, 
                                       #AsySVDRec, 
                                       #PureSVDRec, 
                                       #PureSVDItemRec, 
                                       #ScaledPureSVDRec, 
                                       #SVDFeatureRec
                                       ], 
                                      dataset_version, 
                                      load_scores_from_saved=True)

0.04631528759395268
0.047454851309241375
0.05114140484470601
[0.0511414  0.04745485 0.04631529] [<class 'recmodels.EASE_R_Rec'> <class 'recmodels.RP3BetaRec'>
 <class 'recmodels.P3AlphaRec'>]


In [10]:
optimizer.incremental_bayesian_search(100, 0.3)

EvaluatorHoldout: Ignoring 31 ( 0.1%) Users that have less than 1 test interactions
Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'w0': 2.9801817397785006, 'w1': 6.3770998517711055, 'normalize': '-inf'}
DiffStructHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
EvaluatorHoldout: Processed 41598 (100.0%) in 47.37 sec. Users per second: 878
SearchBayesianSkopt: New best config found. Config 0: {'w0': 2.9801817397785006, 'w1': 6.3770998517711055, 'normalize': '-inf'} - results: PRECISION: 0.0940839, PRECISION_RECALL_MIN_DEN: 0.1050949, RECALL: 0.0755003, MAP: 0.0510544, MAP_MIN_DEN: 0.0564097, MRR: 0.2695388, NDCG: 0.1173261, F1: 0.0837739, HIT_RATE: 0.4919227, ARHR_ALL_HITS: 0.3648739, NOVELTY: 0.0038176, AVERAGE_POPULARITY: 0.1985722, DIVERSITY_MEAN_INTER_LIST: 0.9395908, DIVERSITY_HERFINDAHL: 0.9939568, COVERAGE_ITEM: 0.5403318, COVERAGE_ITEM_HIT: 0.0547411, ITEMS_IN_GT: 0.8762514, COVERAGE_USER: 0.9992553, C

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 32.2021
Function value obtained: -0.0484
Current minimum: -0.0515
Iteration No: 7 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'w0': 3.5179917822106606, 'w1': 7.934508459689438, 'normalize': 'inf'}
DiffStructHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
EvaluatorHoldout: Processed 41598 (100.0%) in 45.87 sec. Users per second: 907
SearchBayesianSkopt: Config 6 is suboptimal. Config: {'w0': 3.5179917822106606, 'w1': 7.934508459689438, 'normalize': 'inf'} - results: PRECISION: 0.0935598, PRECISION_RECALL_MIN_DEN: 0.1045673, RECALL: 0.0751975, MAP: 0.0506713, MAP_MIN_DEN: 0.0559833, MRR: 0.2681067, NDCG: 0.1166339, F1: 0.0833796, HIT_RATE: 0.4901438, ARHR_ALL_HITS: 0.3626087, NOVELTY: 0.0038312, AVERAGE_POPULARITY: 0.2002710, DIVERSITY_MEAN_INTER_LIST: 0.9374596, DIVERSITY_HERFINDAHL: 0.9937437, COVERAGE_ITEM: 0.5831307, COVERAGE_ITEM_HIT: 0.0542763, ITEMS_

EvaluatorHoldout: Processed 41598 (100.0%) in 30.89 sec. Users per second: 1347
SearchBayesianSkopt: Config 12 is suboptimal. Config: {'w0': 1.8913199354024064, 'w1': 7.1309479399942335, 'normalize': None} - results: PRECISION: 0.0939757, PRECISION_RECALL_MIN_DEN: 0.1049847, RECALL: 0.0754215, MAP: 0.0509833, MAP_MIN_DEN: 0.0563277, MRR: 0.2691894, NDCG: 0.1171822, F1: 0.0836824, HIT_RATE: 0.4915381, ARHR_ALL_HITS: 0.3643935, NOVELTY: 0.0038201, AVERAGE_POPULARITY: 0.1988851, DIVERSITY_MEAN_INTER_LIST: 0.9391984, DIVERSITY_HERFINDAHL: 0.9939176, COVERAGE_ITEM: 0.5490561, COVERAGE_ITEM_HIT: 0.0546696, ITEMS_IN_GT: 0.8762514, COVERAGE_USER: 0.9992553, COVERAGE_USER_HIT: 0.4911720, USERS_IN_GT: 0.9992553, DIVERSITY_GINI: 0.0407891, SHANNON_ENTROPY: 9.2229282, RATIO_DIVERSITY_HERFINDAHL: 0.9942460, RATIO_DIVERSITY_GINI: 0.0921386, RATIO_SHANNON_ENTROPY: 0.6836736, RATIO_AVERAGE_POPULARITY: 3.9356297, RATIO_NOVELTY: 0.3029576, 

Iteration No: 13 ended. Evaluation done at random point.
Time 

EvaluatorHoldout: Processed 41598 (100.0%) in 42.06 sec. Users per second: 989
SearchBayesianSkopt: Config 18 is suboptimal. Config: {'w0': 3.7290358973590005, 'w1': 6.111417674115942, 'normalize': 'inf'} - results: PRECISION: 0.0940069, PRECISION_RECALL_MIN_DEN: 0.1050157, RECALL: 0.0754527, MAP: 0.0509987, MAP_MIN_DEN: 0.0563471, MRR: 0.2692200, NDCG: 0.1172153, F1: 0.0837140, HIT_RATE: 0.4916102, ARHR_ALL_HITS: 0.3644780, NOVELTY: 0.0038196, AVERAGE_POPULARITY: 0.1988097, DIVERSITY_MEAN_INTER_LIST: 0.9392882, DIVERSITY_HERFINDAHL: 0.9939266, COVERAGE_ITEM: 0.5476259, COVERAGE_ITEM_HIT: 0.0546696, ITEMS_IN_GT: 0.8762514, COVERAGE_USER: 0.9992553, COVERAGE_USER_HIT: 0.4912441, USERS_IN_GT: 0.9992553, DIVERSITY_GINI: 0.0405844, SHANNON_ENTROPY: 9.2214635, RATIO_DIVERSITY_HERFINDAHL: 0.9942550, RATIO_DIVERSITY_GINI: 0.0916764, RATIO_SHANNON_ENTROPY: 0.6835650, RATIO_AVERAGE_POPULARITY: 3.9341382, RATIO_NOVELTY: 0.3029220, 

Iteration No: 19 ended. Evaluation done at random point.
Time t

EvaluatorHoldout: Processed 41598 (100.0%) in 47.43 sec. Users per second: 877
SearchBayesianSkopt: Config 25 is suboptimal. Config: {'w0': 4.888472373371943, 'w1': 8.716516334781485, 'normalize': 'inf'} - results: PRECISION: 0.0938579, PRECISION_RECALL_MIN_DEN: 0.1048711, RECALL: 0.0753689, MAP: 0.0509150, MAP_MIN_DEN: 0.0562537, MRR: 0.2690436, NDCG: 0.1170692, F1: 0.0836034, HIT_RATE: 0.4914419, ARHR_ALL_HITS: 0.3640877, NOVELTY: 0.0038225, AVERAGE_POPULARITY: 0.1992361, DIVERSITY_MEAN_INTER_LIST: 0.9387810, DIVERSITY_HERFINDAHL: 0.9938758, COVERAGE_ITEM: 0.5571010, COVERAGE_ITEM_HIT: 0.0544193, ITEMS_IN_GT: 0.8762514, COVERAGE_USER: 0.9992553, COVERAGE_USER_HIT: 0.4910759, USERS_IN_GT: 0.9992553, DIVERSITY_GINI: 0.0418831, SHANNON_ENTROPY: 9.2309465, RATIO_DIVERSITY_HERFINDAHL: 0.9942042, RATIO_DIVERSITY_GINI: 0.0946100, RATIO_SHANNON_ENTROPY: 0.6842680, RATIO_AVERAGE_POPULARITY: 3.9425772, RATIO_NOVELTY: 0.3031463, 

Iteration No: 26 ended. Evaluation done at random point.
Time ta

...

Iteration No: 79 ended. Search finished for the next optimal point.
Time taken: 67.5171
Function value obtained: -0.0507
Current minimum: -0.0516
Iteration No: 80 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'w0': 5.453631021511987, 'w1': 1.1893449149019644, 'normalize': 'fro'}
DiffStructHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
DiffStructHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
DiffStructHybridRecommender: Loading model from file 'recommendations/interactions-all-ones/DiffStructHybridRecommender/version/optimizationDiffStructHybridRecommender_best_model.zip'
DiffStructHybridRecommender: Loading complete
DiffStructHybridRecommender: Saving item_score in file 'recommendations/interactions-all-ones/DiffStructHybridRecommender/version/item_scores.npy'
DiffStructHybridRecommender: Saving complete
EvaluatorHoldout: Processed 41598 (100.0%) in 44.29 sec. Users per second: 939
SearchBayesian

DiffStructHybridRecommender: Saving complete
EvaluatorHoldout: Processed 41598 (100.0%) in 43.61 sec. Users per second: 954
SearchBayesianSkopt: Config 83 is suboptimal. Config: {'w0': 6.672769602782445, 'w1': 0.1357705263174336, 'normalize': 'fro'} - results: PRECISION: 0.0950238, PRECISION_RECALL_MIN_DEN: 0.1059615, RECALL: 0.0758189, MAP: 0.0515879, MAP_MIN_DEN: 0.0569209, MRR: 0.2706065, NDCG: 0.1181721, F1: 0.0843419, HIT_RATE: 0.4931006, ARHR_ALL_HITS: 0.3672295, NOVELTY: 0.0037803, AVERAGE_POPULARITY: 0.1935963, DIVERSITY_MEAN_INTER_LIST: 0.9453048, DIVERSITY_HERFINDAHL: 0.9945282, COVERAGE_ITEM: 0.3196510, COVERAGE_ITEM_HIT: 0.0552775, ITEMS_IN_GT: 0.8762514, COVERAGE_USER: 0.9992553, COVERAGE_USER_HIT: 0.4927334, USERS_IN_GT: 0.9992553, DIVERSITY_GINI: 0.0226110, SHANNON_ENTROPY: 9.0549127, RATIO_DIVERSITY_HERFINDAHL: 0.9948568, RATIO_DIVERSITY_GINI: 0.0510762, RATIO_SHANNON_ENTROPY: 0.6712190, RATIO_AVERAGE_POPULARITY: 3.8309741, RATIO_NOVELTY: 0.2998028, 

Iteration No: 84 e

Iteration No: 88 ended. Search finished for the next optimal point.
Time taken: 64.7614
Function value obtained: -0.0487
Current minimum: -0.0516
Iteration No: 89 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'w0': 5.601422822473295, 'w1': 5.553083345796141, 'normalize': 'fro'}
DiffStructHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
DiffStructHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
DiffStructHybridRecommender: Loading model from file 'recommendations/interactions-all-ones/DiffStructHybridRecommender/version/optimizationDiffStructHybridRecommender_best_model.zip'
DiffStructHybridRecommender: Loading complete
DiffStructHybridRecommender: Saving item_score in file 'recommendations/interactions-all-ones/DiffStructHybridRecommender/version/item_scores.npy'
DiffStructHybridRecommender: Saving complete
EvaluatorHoldout: Processed 41598 (100.0%) in 43.09 sec. Users per second: 965
SearchBayesianS

DiffStructHybridRecommender: Saving complete
EvaluatorHoldout: Processed 41598 (100.0%) in 52.79 sec. Users per second: 788
SearchBayesianSkopt: Config 92 is suboptimal. Config: {'w0': 0.7679078173005461, 'w1': 1.4700959769640833, 'normalize': 'fro'} - results: PRECISION: 0.0915789, PRECISION_RECALL_MIN_DEN: 0.1020437, RECALL: 0.0731416, MAP: 0.0488737, MAP_MIN_DEN: 0.0540053, MRR: 0.2595859, NDCG: 0.1132988, F1: 0.0813284, HIT_RATE: 0.4821626, ARHR_ALL_HITS: 0.3505136, NOVELTY: 0.0037956, AVERAGE_POPULARITY: 0.2060219, DIVERSITY_MEAN_INTER_LIST: 0.9312223, DIVERSITY_HERFINDAHL: 0.9931200, COVERAGE_ITEM: 0.4501216, COVERAGE_ITEM_HIT: 0.0441219, ITEMS_IN_GT: 0.8762514, COVERAGE_USER: 0.9992553, COVERAGE_USER_HIT: 0.4818036, USERS_IN_GT: 0.9992553, DIVERSITY_GINI: 0.0335743, SHANNON_ENTROPY: 8.9861059, RATIO_DIVERSITY_HERFINDAHL: 0.9934482, RATIO_DIVERSITY_GINI: 0.0758411, RATIO_SHANNON_ENTROPY: 0.6661185, RATIO_AVERAGE_POPULARITY: 4.0768560, RATIO_NOVELTY: 0.3010181, 

Iteration No: 93 

Iteration No: 97 ended. Search finished for the next optimal point.
Time taken: 70.8345
Function value obtained: -0.0491
Current minimum: -0.0516
Iteration No: 98 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'w0': 2.2512568397085757, 'w1': 0.0, 'normalize': 'fro'}
DiffStructHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
DiffStructHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
DiffStructHybridRecommender: Loading model from file 'recommendations/interactions-all-ones/DiffStructHybridRecommender/version/optimizationDiffStructHybridRecommender_best_model.zip'
DiffStructHybridRecommender: Loading complete
DiffStructHybridRecommender: Saving item_score in file 'recommendations/interactions-all-ones/DiffStructHybridRecommender/version/item_scores.npy'
DiffStructHybridRecommender: Saving complete
EvaluatorHoldout: Processed 41598 (100.0%) in 51.99 sec. Users per second: 800
SearchBayesianSkopt: Config 

In [13]:
kwargs = {"load_scores_from_saved": True, 
          "recs_on_urm_splitted": False, 
          "user_id_array_val": utils.get_users_for_submission(), 
          "new_item_scores_file_name_root": "on_all_urm_"}

rec = utils.load_best_model(URM_all, 
                      recommender_class, 
                      dataset_version, 
                      True,
                      **kwargs)

DiffStructHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
DiffStructHybridRecommender: Loading model from file 'recommendations/interactions-all-ones/DiffStructHybridRecommender/version/optimizationDiffStructHybridRecommender_best_model.zip'
DiffStructHybridRecommender: Loading complete


In [15]:
utils.submission(rec, dataset_version, override=True)

DiffStructHybridRecommender: Saving model in file 'recommendations/interactions-all-ones/DiffStructHybridRecommender/version/bestDiffStructHybridRecommender'
DiffStructHybridRecommender: Saving complete
